In [ ]:
from openai import OpenAI
from IPython.display import Markdown, display
from playwright.async_api import async_playwright

In [2]:
openai = OpenAI()
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [3]:
system_prompt = """
    You are a helpful assistant. You answer exactly to the user's specifications in a concise manner.
"""

user_prompt_prefix = """
    Provide a summary of the following html content in markdown:
"""

In [4]:
async def fetch_website_html(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)
        await page.wait_for_load_state('networkidle');
        html = await page.content()
        await browser.close()
        return html


def messages_for(website_html):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website_html}
    ]


async def summarize(url):
    website_html = await fetch_website_html(url)
    response = ollama.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website_html)
    )
    return response.choices[0].message.content


async def display_summary(url):
    summary = await summarize(url)
    display(Markdown(summary))

In [5]:
await display_summary("https://edwarddonner.com/")

APIConnectionError: Connection error.